Парсер сайта "Реформа ЖКХ" https://www.reformagkh.ru/

Жилые дома Москвы. Часть 1. Собирает данные со страниц муниципальных районов:

- адрес дома
- год ввода в эксплуатацию
- общая плошадь (кв.м.)
- управляющая организация
- ссылка на страницу дома

Последнее обновление парсера: 2021-07-31

### Собираю ссылки на муниципальные районы ###

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)

Получаю ссылки на муниципальные районы

In [ ]:
# ссылка на гланую страницу
main_url = 'https://www.reformagkh.ru/myhouse?tid=2280999'

In [ ]:
driver.get(main_url)
sleep(5) # даю странице "прогрузиться"

In [ ]:
# "раскрываю" подкатегории
plus_list = driver.find_elements_by_tag_name('svg[style="cursor: pointer"]') # список "плюсов"

driver.implicitly_wait(10)

for i in plus_list: # раскрываю все списки муниципалитетов
    i.click()

In [ ]:
# вытаскиваю ссылки

data = [] # список для данных

# вытаскиваю ссылки
mun_urls = driver.find_elements_by_class_name('d-flex.justify-content-between.align-items-top.my-3')

# циклом вытаскиваю ссылки и количество домов в каждом муниципалитете 
for mun in mun_urls:
    mun_districts = mun.text
    
    try:
        page_url = mun.find_element_by_tag_name('a').get_attribute('href')
    
    except:
        page_url = 'none'
        
    data.append([mun_districts, page_url])

In [ ]:
# создаю датафрем с данными

head_list = ['mun_districts', 'page_url']
work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.sample(3)

In [ ]:
# должно быть 165 районов: это районы + административные округа
work.shape

Ссылки на административные районы не нужны, это подкатегории, там только ссылки. Поэтому их убираю.

Точнее, оставляю только ссылки на муниципалитеты. У этих страниц есть "хвост"  — "&item=tp".

Вместе с адм.округами отваливается "Воскресенское", поскольку домов по версии "Реформы"  там — 0.

In [ ]:
# отсеиваю лишнее

work = work[work.page_url.str.contains('item=tp', case=False)]

In [ ]:
work.sample(3)

In [ ]:
# разбиваю столбец mun_districts на 2 столбца

work[['mun_districts','house_number']] = work['mun_districts'].str.split('\n', expand=True)

In [ ]:
work = work[['mun_districts', 'house_number', 'page_url']]

In [ ]:
# Записываю в файл (на всякий)

work.to_excel('C:/00 Данные/Разные Москва/mun_urls.xlsx', encoding='utf-8', index=False)

## Собираю ссылки на страницы домов ##

In [ ]:
# Обрезаю ссылки

work['page_url'] = work['page_url'].str.replace('&item=tp', '')

In [ ]:
# список для данных

data = []

In [ ]:
for district_page in tqdm(range(work.shape[0])):
    
    next_pages = math.ceil(int(work.iloc[district_page][1]) / 60) + 1   
       
    for i in range(1, next_pages):
        
        url = work.iloc[district_page][2] + '&page=' + str(i) + '&limit=60&item'
        
        driver.get(url)
        sleep(5) # пауза, чтобы страница прогрузилась
        driver.implicitly_wait(5)
               
        # собираю данные со страницы
        # Данные на сайте сверстаны таблицей, поэтому использую xpath
        
        page_building_number = len(driver.find_element_by_tag_name('tbody').find_elements_by_class_name('text-right')) + 1
        
        for n in range(1, page_building_number): # цикл для сбора с             
            address = driver.find_element_by_xpath(F'/html/body/section[5]/div/table/tbody/tr[{n}]/td[1]/a').text
            year_exp = driver.find_element_by_xpath(F'/html/body/section[5]/div/table/tbody/tr[{n}]/td[2]').text
            square = driver.find_element_by_xpath(F'/html/body/section[5]/div/table/tbody/tr[{n}]/td[3]').text
            managing_organization = driver.find_element_by_xpath(F'/html/body/section[5]/div/table/tbody/tr[{n}]/td[4]').text
            mun_district = work.iloc[district_page][0]
            building_page = driver.find_element_by_xpath(F'/html/body/section[5]/div/table/tbody/tr[{n}]/td[1]/a').get_attribute('href')
            
            data.append([address, year_exp, square, managing_organization, building_page, mun_district])

In [ ]:
head_list = ['address', 'year_exp', 'square', 'managing_organization', 'mun_district', 'building_page']
work_links = pd.DataFrame(data, columns=head_list)

In [ ]:
work_links.sample(3)

In [ ]:
work_links.shape

In [ ]:
# проверяю все ли данные скачались
# количество строк должно быть равно сумме всех домов

work_links.shape[0] == work.house_number.sum()

In [ ]:
# Записываю в файл

work_links.to_excel('C:/00 Данные/Разные Москва/buildings_links.xlsx', encoding='utf-8', index=False)